In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import librosa as lb
import glob
from pprint import pprint as pp
import scipy
import scipy.signal.windows as wn 
import signal as sp
import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (14,5)

from  more_itertools import unique_everseen
from collections import OrderedDict
# import soundfile as sf
# import difflib
import statistics
import itertools
from scipy.stats import skew
from scipy.stats import kurtosis

In [2]:
import glob
data ={}
name_list =[]
path = './songs/'

files = [f for f in glob.glob(path + "**/*.wav", recursive=True)]
files += [f for f in glob.glob(path + "**/*.mp3", recursive=True)]

for file in files:
    name = file.split('/')[-1].split('.')[0]
    y, sr = lb.load(file, sr=44100)
    data[name] = {'y' : y, 'sr': sr}

In [3]:
def get_features_mean(song, sr, hop_length, n_fft):
    # try:
    print('extracting features...')
    y_harmonic, y_percussive = lb.effects.hpss(song)  # split song into harmonic and percussive parts
    stft_harmonic = lb.core.stft(y_harmonic, n_fft=n_fft, hop_length=hop_length)  # Compute power spectrogram.
    stft_percussive = lb.core.stft(y_percussive, n_fft=n_fft, hop_length=hop_length)  # Compute power spectrogram.

    rmsH = np.sqrt(np.mean(np.abs(lb.feature.rms(S=stft_harmonic)) ** 2, axis=0, keepdims=True))
    rmsH_a = np.mean(rmsH)
    rmsH_std = np.std(rmsH)
    rmsH_skew = skew(np.mean(rmsH, axis=0))
    rmsH_kurtosis = kurtosis(np.mean(rmsH, axis=0), fisher=True, bias=True)

    rmsP = np.sqrt(np.mean(np.abs(lb.feature.rms(S=stft_percussive)) ** 2, axis=0, keepdims=True))
    rmsP_a = np.mean(rmsP)
    rmsP_std = np.std(rmsP)
    rmsP_skew = skew(np.mean(rmsP, axis=0))
    rmsP_kurtosis = kurtosis(np.mean(rmsP, axis=0), fisher=True, bias=True)

    centroid = lb.feature.spectral_centroid(y=song, sr=sr, n_fft=n_fft, hop_length=hop_length) # ) # Compute the spectral centroid.
    centroid_a = np.mean(centroid)
    centroid_std = np.std(centroid)

    bw = lb.feature.spectral_bandwidth(y=song, sr=sr, n_fft=n_fft,
                                       hop_length=hop_length)  # Compute p’th-order spectral bandwidth:
    bw_a = np.mean(bw)
    bw_std = np.std(bw)

    contrast = lb.feature.spectral_contrast(y=song, sr=sr, n_fft=n_fft,
                                            hop_length=hop_length)  # Compute spectral contrast [R16]
    contrast_a = np.mean(contrast)
    contrast_std = np.std(contrast)

    polyfeat = lb.feature.poly_features(y=y_harmonic,sr=sr, n_fft=n_fft,
                                        hop_length=hop_length)  # Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram.
    polyfeat_a = np.mean(polyfeat[0])
    polyfeat_std = np.std(polyfeat[0])

    tonnetz = lb.feature.tonnetz(y=lb.effects.harmonic(y=y_harmonic), sr=sr)  # Computes the tonal centroid features (tonnetz), following the method of [R17].
    tonnetz_a = np.mean(tonnetz)
    tonnetz_std = np.std(tonnetz)

    zcr = lb.feature.zero_crossing_rate(song, hop_length=hop_length)  # zero crossing rate
    zcr_a = np.mean(zcr)
    zcr_std = np.std(zcr)

    onset_env = lb.onset.onset_strength(y=y_percussive, sr=sr)
    onset_a = np.mean(onset_env)
    onset_std = np.std(onset_env)

    D = lb.stft(song)
    times = lb.frames_to_time(np.arange(D.shape[1]))  # not returned, but could be if you want to plot things as a time series

    bpm, beats = lb.beat.beat_track(y=y_percussive, sr=sr, onset_envelope=onset_env, units='time')
    beats_a = np.mean(beats)
    beats_std = np.std(beats)

    features_dict = OrderedDict({'rmseP_a': rmsP_a, 'rmseP_std': rmsP_std, 'rmseH_a': rmsH_a, 'rmseH_std': rmsH_std,
                                 'centroid_a': centroid_a, 'centroid_std': centroid_std, 'bw_a': bw_a, 'bw_std': bw_std,
                                 'contrast_a': contrast_a, 'contrast_std': contrast_std, 'polyfeat_a': polyfeat_a,
                                 'polyfeat_std': polyfeat_std, 'tonnetz_a': tonnetz_a, 'tonnetz_std': tonnetz_std,
                                 'zcr_a': zcr_a, 'zcr_std': zcr_std, 'onset_a': onset_a, 'onset_std': onset_std,
                                 'bpm': bpm, 'rmseP_skew': rmsP_skew, 'rmseP_kurtosis': rmsP_kurtosis,
                                 'rmseH_skew': rmsH_skew, 'rmseH_kurtosis': rmsH_kurtosis, 'beats_a':beats_a,
                                 'beats_std':beats_std})

    # combine_features = {**features_dict, **bands_dict}
    print('features extracted successfully')
    return features_dict

In [4]:
data.keys()

dict_keys(['songs\\electro house\\Alex Gaudino & Shena - Watch Out', 'songs\\afro house\\Aero Manyelo, Thee Suka, Nkuli Keys - Indlela (radio edit)', 'songs\\afro house\\Afrika System - Anikana-O (Massimo Berardi Extended Remix)', 'songs\\afro house\\Afro Wav, K', 'songs\\deep melodic house\\Anarcrusan - In My Mind Armonica Rework Yoshitoshi Recordings-www', 'songs\\deep melodic house\\Krister Linder Sasha - Cut Me Down feat', 'songs\\electro house\\Rahpael Mader - Vacuity ', 'songs\\melodic house\\26', 'songs\\melodic house\\Clay Smith - Metronome (Original Mix) [Clinique Recordings]'])

In [5]:
#res = get_features_mean(data['songs\\Alex Gaudino & Shena - Watch Out']['y'], data['songs\\Alex Gaudino & Shena - Watch Out']['sr'], hop_length=512, n_fft=2048)
results = {}  # create an empty dictionary to store the results

for key in data.keys():
    features = get_features_mean(data[key]['y'], data[key]['sr'], hop_length=512, n_fft=2048)
    results[key] = features

# print the results
for key, features in results.items():
    print(f"Features for {key}:")
    print(features)
    print()
    
# Convert the results dictionary to a pandas DataFrame
df = pd.DataFrame.from_dict(results, orient='index')

# Save the DataFrame to a CSV file
df.to_csv('./data/features_results.csv', index_label='Song')

print("CSV file created successfully.")


extracting features...
features extracted successfully
extracting features...
features extracted successfully
extracting features...
features extracted successfully
extracting features...
features extracted successfully
extracting features...
features extracted successfully
extracting features...
features extracted successfully
extracting features...
features extracted successfully
extracting features...
features extracted successfully
extracting features...
features extracted successfully
Features for songs\electro house\Alex Gaudino & Shena - Watch Out:
OrderedDict({'rmseP_a': 0.05234401991024035, 'rmseP_std': 0.03704689411844411, 'rmseH_a': 0.1308240310333169, 'rmseH_std': 0.06291546229762582, 'centroid_a': 3975.8341038315584, 'centroid_std': 1240.0806328446129, 'bw_a': 4021.5593733230685, 'bw_std': 984.9219943850235, 'contrast_a': 18.015253903993905, 'contrast_std': 9.118587993697401, 'polyfeat_a': -0.00023687952872098725, 'polyfeat_std': 7.988136079842154e-05, 'tonnetz_a': -0.0070

In [6]:
results

{'songs\\electro house\\Alex Gaudino & Shena - Watch Out': OrderedDict([('rmseP_a',
               0.05234401991024035),
              ('rmseP_std', 0.03704689411844411),
              ('rmseH_a', 0.1308240310333169),
              ('rmseH_std', 0.06291546229762582),
              ('centroid_a', 3975.8341038315584),
              ('centroid_std', 1240.0806328446129),
              ('bw_a', 4021.5593733230685),
              ('bw_std', 984.9219943850235),
              ('contrast_a', 18.015253903993905),
              ('contrast_std', 9.118587993697401),
              ('polyfeat_a', -0.00023687952872098725),
              ('polyfeat_std', 7.988136079842154e-05),
              ('tonnetz_a', -0.007057023459117997),
              ('tonnetz_std', 0.08704874271459756),
              ('zcr_a', 0.07851646658941046),
              ('zcr_std', 0.046901517448203735),
              ('onset_a', 1.6695093),
              ('onset_std', 1.7830656),
              ('bpm', 129.19921875),
              ('